In [1]:
# ===============================
# 📦 Imports and setup
# ===============================
import os
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import Window
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# ===============================
# ⚙️ Initialize Spark
# ===============================
def initialize_spark():
    spark = pyspark.sql.SparkSession.builder \
        .appName("gold_feature_store_daily") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .master("local[*]") \
        .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

In [3]:
# ===============================
# 📥 Load Silver Tables
# ===============================
def load_tables(spark):
    print("Loading Silver layer tables...\n")

    df_userlogs = spark.read.parquet("/app/datamart/silver/user_logs")
    df_transactions = spark.read.parquet("/app/datamart/silver/transactions")
    df_latest_transactions = spark.read.parquet("/app/datamart/silver/latest_transactions")
    df_members = spark.read.parquet("/app/datamart/silver/members")

    return df_userlogs, df_transactions, df_latest_transactions, df_members

In [4]:
# ===============================
# 👤 Registered Users
# ===============================
def get_registered_users(df_members, inference_date):
    registered_users = (
        df_members
        .withColumn("registration_date", F.to_date("registration_date"))
        .filter(F.col("registration_date") <= F.to_date(F.lit(inference_date)))
        .withColumn(
            "tenure_days_at_snapshot",
            F.datediff(F.to_date(F.lit(inference_date)), F.col("registration_date"))
        )
        .select("msno", "registration_date", "tenure_days_at_snapshot",
                "registered_via", "city_clean")
    )

    registered_users = (
        registered_users
        .withColumn("registered_via_oh", F.coalesce(F.col("registered_via"), F.lit(0)))
        .withColumn("city_clean_oh", F.coalesce(F.col("city_clean"), F.lit(0)))
        .select("msno", "registration_date", "tenure_days_at_snapshot",
                "registered_via_oh", "city_clean_oh")
    )

    print(f"📄 Registered users up to {inference_date}: {registered_users.count()}")
    return registered_users

In [5]:
# ===============================
# 🎧 Filter User Logs by Date Windows
# ===============================
def filter_userlogs(df_userlogs, inference_date):
    ref_today = F.to_date(F.lit(inference_date))
    lower30 = F.date_sub(ref_today, 29)
    lower7 = F.date_sub(ref_today, 6)

    df_userlogs = df_userlogs.withColumn("date", F.to_date("date"))

    userlogs_30d = df_userlogs.filter((F.col("date") >= lower30) & (F.col("date") <= ref_today))
    userlogs_7d = df_userlogs.filter((F.col("date") >= lower7) & (F.col("date") <= ref_today))

    return userlogs_30d, userlogs_7d

In [6]:
# ===============================
# 🧮 User Log Features
# ===============================
def add_userlog_features(registered_users, userlogs_30d, userlogs_7d, df_userlogs, inference_date):
    # Sum seconds (30d)
    user_sum_30d = userlogs_30d.groupBy("msno").agg(F.sum("total_secs").alias("sum_secs_w30"))
    registered_users = registered_users.join(user_sum_30d, "msno", "left").na.fill({"sum_secs_w30": 0.0})

    # Active days (30d)
    user_active_days_30d = userlogs_30d.groupBy("msno").agg(F.countDistinct("date").alias("active_days_w30"))
    registered_users = registered_users.join(user_active_days_30d, "msno", "left").na.fill({"active_days_w30": 0})

    # Completion rate (30d)
    user_complete_rate_30d = userlogs_30d.groupBy("msno").agg(
        (F.sum("num_100") / F.when(F.sum("num_unq") > 0, F.sum("num_unq")).otherwise(F.lit(1))).alias("complete_rate_w30")
    )
    registered_users = registered_users.join(user_complete_rate_30d, "msno", "left").na.fill({"complete_rate_w30": 0.0})

    # Sum seconds (7d)
    user_sum_7d = userlogs_7d.groupBy("msno").agg(F.sum("total_secs").alias("sum_secs_w7"))
    registered_users = registered_users.join(user_sum_7d, "msno", "left").na.fill({"sum_secs_w7": 0.0})

    # Engagement ratio
    registered_users = registered_users.withColumn(
        "engagement_ratio_7_30",
        F.col("sum_secs_w7") / F.when(F.col("sum_secs_w30") > 0, F.col("sum_secs_w30")).otherwise(F.lit(1))
    )

    # Days since last play
    last_play = (
        df_userlogs.filter(F.col("date") <= F.lit(inference_date))
        .groupBy("msno")
        .agg(F.max("date").alias("last_play_date"))
    )
    registered_users = registered_users.join(last_play, "msno", "left") \
        .withColumn("days_since_last_play", F.datediff(F.lit(inference_date), F.col("last_play_date")))

    # Trend in total_secs (30d)
    daily_secs = userlogs_30d.groupBy("msno", "date").agg(F.sum("total_secs").alias("daily_secs"))
    window_spec = Window.partitionBy("msno").orderBy("date")
    daily_secs = daily_secs.withColumn("day_idx", F.row_number().over(window_spec))
    trend = daily_secs.groupBy("msno").agg(
        (F.covar_pop("day_idx", "daily_secs") / F.var_pop("day_idx")).alias("trend_secs_w30")
    )
    registered_users = registered_users.join(trend, "msno", "left").na.fill({"trend_secs_w30": 0.0})

    return registered_users

In [7]:
# ===============================
# 💳 Transaction Features
# ===============================
def add_transaction_features(registered_users, df_transactions, df_latest_transactions, inference_date):
    df_transactions_filtered = df_transactions.filter(
        F.to_date(F.col("transaction_date")) <= F.to_date(F.lit(inference_date))
    )

    registered_users = registered_users.join(df_latest_transactions, "msno", "left") \
        .withColumn("tenure_days", F.datediff(F.col("transaction_date"), F.col("registration_date"))) \
        .na.fill({"tenure_days": 0})

    auto_renew_stats = (
        df_transactions_filtered.groupBy("msno")
        .agg(
            F.sum(F.when(F.col("is_auto_renew") == 1, 1).otherwise(0)).alias("auto_renew_count"),
            F.count("*").alias("total_tx_before_expire")
        )
        .withColumn(
            "auto_renew_share",
            F.col("auto_renew_count") / F.when(F.col("total_tx_before_expire") > 0, F.col("total_tx_before_expire")).otherwise(F.lit(1))
        )
    )
    registered_users = registered_users.join(auto_renew_stats.select("msno", "auto_renew_share"), "msno", "left") \
        .na.fill({"auto_renew_share": 0.0})

    last_is_auto_renew = df_latest_transactions.select("msno", "is_auto_renew").withColumnRenamed("is_auto_renew", "last_is_auto_renew")
    registered_users = registered_users.join(last_is_auto_renew, "msno", "left")

    last_list_price = df_latest_transactions.select("msno", "plan_list_price").withColumnRenamed("plan_list_price", "last_plan_list_price")
    registered_users = registered_users.join(last_list_price, "msno", "left")

    return registered_users

In [ ]:
# ===============================
# 🧱 Create Gold Features
# ===============================
def create_gold_features(inference_date, spark):
    print(f"\n{'='*60}")
    print(f"🗓️ Creating Gold Feature Store snapshot for: {inference_date}")
    print(f"{'='*60}\n")

    output_path = f"datamart/gold/inference_feature_store/{inference_date.replace('-', '_')}"

    if os.path.exists(output_path):
        print(f"🟡 Skipping {inference_date} — already exists.\n")
        return

    df_userlogs, df_transactions, df_latest_transactions, df_members = load_tables(spark)

    registered_users = get_registered_users(df_members, inference_date)
    userlogs_30d, userlogs_7d = filter_userlogs(df_userlogs, inference_date)
    registered_users = add_userlog_features(registered_users, userlogs_30d, userlogs_7d, df_userlogs, inference_date)
    registered_users = add_transaction_features(registered_users, df_transactions, df_latest_transactions, inference_date)
    registered_users = registered_users.na.fill(0)

    registered_users.write.mode("overwrite").parquet(output_path)
    print(f"✅ Features saved to {output_path}")
    print(f"📊 Total records: {registered_users.count()}\n")

In [9]:
# ===============================
# 🚀 Main: Generate Daily Snapshots
# ===============================
def main():
    spark = initialize_spark()

    start_date = datetime(2015, 2, 1)
    end_date = datetime(2015, 2, 3)

    print(f"\n{'='*80}")
    print(f"🚀 Generating DAILY Gold Feature Store snapshots")
    print(f"📆 Date range: {start_date.strftime('%Y-%m-%d')} → {end_date.strftime('%Y-%m-%d')}")
    print(f"{'='*80}\n")

    current = start_date
    total_days = 0

    while current <= end_date:
        inference_date = current.strftime("%Y-%m-%d")
        print(f"\n📅 Snapshot for {inference_date}")
        try:
            create_gold_features(inference_date, spark)
            total_days += 1
        except Exception as e:
            print(f"⚠️ Failed for {inference_date}: {e}")

        current += timedelta(days=1)

    print(f"\n✅ Completed all daily feature stores.")
    print(f"Total snapshots created: {total_days}")

    spark.stop()

# Run main
main()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/02 10:29:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/02 10:29:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/02 10:29:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.



🚀 Generating DAILY Gold Feature Store snapshots
📆 Date range: 2015-02-01 → 2015-02-03


📅 Snapshot for 2015-02-01

🗓️ Creating Gold Feature Store snapshot for: 2015-02-01

Loading Silver layer tables...

📄 Registered users up to 2015-02-01: 458


✅ Features saved to datamart/gold/feature_store/2015_02_01
📊 Total records: 458


📅 Snapshot for 2015-02-02

🗓️ Creating Gold Feature Store snapshot for: 2015-02-02

Loading Silver layer tables...

📄 Registered users up to 2015-02-02: 458


✅ Features saved to datamart/gold/feature_store/2015_02_02
📊 Total records: 458


📅 Snapshot for 2015-02-03

🗓️ Creating Gold Feature Store snapshot for: 2015-02-03

Loading Silver layer tables...

📄 Registered users up to 2015-02-03: 458


✅ Features saved to datamart/gold/feature_store/2015_02_03
📊 Total records: 458


✅ Completed all daily feature stores.
Total snapshots created: 3


import os
os.system("pkill -f 'org.apache.spark.deploy'")  # kill any Spark JVMs